# **Homework 2: Phoneme Classification**


Objectives:
* Solve a classification problem with deep neural networks (DNNs).
* Understand recursive neural networks (RNNs).

If you have any questions, please contact the TAs via TA hours, NTU COOL, or email to mlta-2023-spring@googlegroups.com

# Some Utility Functions
**Fixes random number generator seeds for reproducibility.**

In [1]:
!pip install torchensemble

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 kB 2.6 MB/s eta 0:00:00


In [2]:
import numpy as np
import torch
import random

def same_seeds(seed):
    random.seed(seed) 
    np.random.seed(seed)  
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed) 
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

**Helper functions to pre-process the training data from raw MFCC features of each utterance.**

A phoneme may span several frames and is dependent to past and future frames. \
Hence we concatenate neighboring phonemes for training to achieve higher accuracy. The **concat_feat** function concatenates past and future k frames (total 2k+1 = n frames), and we predict the center frame.

Feel free to modify the data preprocess functions, but **do not drop any frame** (if you modify the functions, remember to check that the number of frames are the same as mentioned in the slides)

In [3]:
import os
import torch
from tqdm import tqdm

def load_feat(path):
    feat = torch.load(path)
    return feat

def shift(x, n):
    if n < 0:
        left = x[0].repeat(-n, 1)
        right = x[:n]
    elif n > 0:
        right = x[-1].repeat(n, 1)
        left = x[n:]
    else:
        return x

    return torch.cat((left, right), dim=0)

def concat_feat(x, concat_n):
    assert concat_n % 2 == 1 # n must be odd
    if concat_n < 2:
        return x
    seq_len, feature_dim = x.size(0), x.size(1)
    x = x.repeat(1, concat_n) 
    x = x.view(seq_len, concat_n, feature_dim).permute(1, 0, 2) # concat_n, seq_len, feature_dim
    mid = (concat_n // 2)
    for r_idx in range(1, mid+1):
        x[mid + r_idx, :] = shift(x[mid + r_idx], r_idx)
        x[mid - r_idx, :] = shift(x[mid - r_idx], -r_idx)

    return x.permute(1, 0, 2).view(seq_len, concat_n * feature_dim)

def preprocess_data(split, feat_dir, phone_path, concat_nframes, train_ratio=0.8):
    class_num = 41 # NOTE: pre-computed, should not need change

    if split == 'train' or split == 'val':
        mode = 'train'
    elif split == 'test':
        mode = 'test'
    else:
        raise ValueError('Invalid \'split\' argument for dataset: PhoneDataset!')

    label_dict = {}
    if mode == 'train':
        for line in open(os.path.join(phone_path, f'{mode}_labels.txt')).readlines():
            line = line.strip('\n').split(' ')
            label_dict[line[0]] = [int(p) for p in line[1:]]
        
        # split training and validation data
        usage_list = open(os.path.join(phone_path, 'train_split.txt')).readlines()
        random.shuffle(usage_list)
        train_len = int(len(usage_list) * train_ratio)
        usage_list = usage_list[:train_len] if split == 'train' else usage_list[train_len:]

    elif mode == 'test':
        usage_list = open(os.path.join(phone_path, 'test_split.txt')).readlines()

    usage_list = [line.strip('\n') for line in usage_list]
    print('[Dataset] - # phone classes: ' + str(class_num) + ', number of utterances for ' + split + ': ' + str(len(usage_list)))

    max_len = 3000000
    X = torch.empty(max_len, 39 * concat_nframes)
    if mode == 'train':
        y = torch.empty(max_len, dtype=torch.long)

    idx = 0
    for i, fname in tqdm(enumerate(usage_list)):
        feat = load_feat(os.path.join(feat_dir, mode, f'{fname}.pt'))
        cur_len = len(feat)
        feat = concat_feat(feat, concat_nframes)
        if mode == 'train':
          label = torch.LongTensor(label_dict[fname])

        X[idx: idx + cur_len, :] = feat
        if mode == 'train':
          y[idx: idx + cur_len] = label

        idx += cur_len

    X = X[:idx, :]
    if mode == 'train':
      y = y[:idx]

    print(f'[INFO] {split} set')
    print(X.shape)
    if mode == 'train':
      print(y.shape)
      return X, y
    else:
      return X


# Dataset

In [4]:
import torch
from torch.utils.data import Dataset

class LibriDataset(Dataset):
    def __init__(self, X, y=None):
        self.data = X
        if y is not None:
            self.label = torch.LongTensor(y)
        else:
            self.label = None

    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]

    def __len__(self):
        return len(self.data)


# Model
Feel free to modify the structure of the model.

In [5]:
import torch.nn as nn

class BasicBlock(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(BasicBlock, self).__init__()

        self.block = nn.Sequential(
            nn.Dropout(0.15),
            nn.Linear(input_dim, output_dim),
            nn.BatchNorm1d(output_dim),
            nn.ReLU(),
        )

    def forward(self, x):
        x = self.block(x)
        return x


class Classifier(nn.Module):
    def __init__(self, input_dim, output_dim=41, hidden_layers=1, hidden_dim=256):
        super(Classifier, self).__init__()

        self.fc = nn.Sequential(
            BasicBlock(input_dim, hidden_dim),
            *[BasicBlock(hidden_dim, hidden_dim) for _ in range(hidden_layers)],
            nn.Linear(hidden_dim, output_dim)
            
            
        )

    def forward(self, x):
        x = self.fc(x)
#         x = nn.functional.softmax(x, dim=1)
        return x

# Hyper-parameters

In [6]:
# data prarameters
concat_nframes = 25      # the number of frames to concat with, n must be odd (total 2k+1 = n frames)
train_ratio = 0.9              # the ratio of data used for training, the rest will be used for validation

# training parameters
seed = 1102                        # random seed
batch_size = 512                # batch size
num_epoch = 20                  # the number of training epoch
learning_rate = 1e-4         # learning rate
model_path = './model.ckpt'     # the path where the checkpoint will be saved

# model parameters
input_dim = 39 * concat_nframes # the input dim of the model, you should not change the value
hidden_layers = 6           # the number of hidden layers
hidden_dim = 700            # the hidden dim

estimators = 5

# Dataloader

In [7]:
from torch.utils.data import DataLoader
import gc

same_seeds(seed)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'DEVICE: {device}')

# preprocess data
train_X, train_y = preprocess_data(split='train', feat_dir='/kaggle/input/ml2023spring-hw2/libriphone/feat', phone_path='/kaggle/input/ml2023spring-hw2/libriphone', concat_nframes=concat_nframes, train_ratio=train_ratio)
val_X, val_y = preprocess_data(split='val', feat_dir='/kaggle/input/ml2023spring-hw2/libriphone/feat', phone_path='/kaggle/input/ml2023spring-hw2/libriphone', concat_nframes=concat_nframes, train_ratio=train_ratio)

# get dataset
train_set = LibriDataset(train_X, train_y)
val_set = LibriDataset(val_X, val_y)

# remove raw feature to save memory
del train_X, train_y, val_X, val_y
gc.collect()

# get dataloader
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

DEVICE: cuda
[Dataset] - # phone classes: 41, number of utterances for train: 3086


3086it [00:24, 124.40it/s]


[INFO] train set
torch.Size([1902169, 975])
torch.Size([1902169])
[Dataset] - # phone classes: 41, number of utterances for val: 343


343it [00:01, 234.83it/s]

[INFO] val set
torch.Size([212222, 975])
torch.Size([212222])


In [8]:
from torchensemble import VotingClassifier  # voting is a classic ensemble strategy
from torchensemble.utils.logging import set_logger

logger = set_logger('log')

ensemble = VotingClassifier(
    estimator=Classifier(input_dim=input_dim, hidden_layers=hidden_layers, hidden_dim=hidden_dim),               # here is your deep learning model
    n_estimators=estimators ,                       # number of base estimators
    cuda=True,
)

criterion = nn.CrossEntropyLoss(label_smoothing = 0.1)   
ensemble.set_criterion(criterion)

ensemble.set_optimizer(
    "AdamW",                                 # type of parameter optimizer
    lr=learning_rate,                       # learning rate of parameter optimizer
)

ensemble.fit(
    train_loader,
    epochs=num_epoch,                          # number of training epochs
    test_loader = val_loader,
    save_dir = './'
)

Log will be saved in '/kaggle/working/logs'.
Create folder 'logs/'
Start logging into file /kaggle/working/logs/log-2023_03_17_04_30.log...
Estimator: 000 | Epoch: 000 | Batch: 000 | Loss: 3.77359 | Correct: 10/512
Estimator: 000 | Epoch: 000 | Batch: 100 | Loss: 2.64725 | Correct: 185/512
Estimator: 000 | Epoch: 000 | Batch: 200 | Loss: 2.34307 | Correct: 210/512
Estimator: 000 | Epoch: 000 | Batch: 300 | Loss: 2.18118 | Correct: 242/512
Estimator: 000 | Epoch: 000 | Batch: 400 | Loss: 2.06105 | Correct: 256/512
Estimator: 000 | Epoch: 000 | Batch: 500 | Loss: 2.04797 | Correct: 268/512
Estimator: 000 | Epoch: 000 | Batch: 600 | Loss: 2.03602 | Correct: 260/512
Estimator: 000 | Epoch: 000 | Batch: 700 | Loss: 2.03355 | Correct: 259/512
Estimator: 000 | Epoch: 000 | Batch: 800 | Loss: 1.97293 | Correct: 266/512
Estimator: 000 | Epoch: 000 | Batch: 900 | Loss: 1.92430 | Correct: 288/512
Estimator: 000 | Epoch: 000 | Batch: 1000 | Loss: 1.87795 | Correct: 288/512
Estimator: 000 | Epoch: 

2023-03-17 04:33:33,873 - INFO: Saving the model to `./VotingClassifier_Classifier_5_ckpt.pth`
2023-03-17 04:33:34,016 - INFO: Epoch: 000 | Validation Acc: 63.675 % | Historical Best: 63.675 %


Estimator: 000 | Epoch: 001 | Batch: 000 | Loss: 1.68488 | Correct: 322/512
Estimator: 000 | Epoch: 001 | Batch: 100 | Loss: 1.80528 | Correct: 311/512
Estimator: 000 | Epoch: 001 | Batch: 200 | Loss: 1.74678 | Correct: 304/512
Estimator: 000 | Epoch: 001 | Batch: 300 | Loss: 1.71058 | Correct: 318/512
Estimator: 000 | Epoch: 001 | Batch: 400 | Loss: 1.67187 | Correct: 319/512
Estimator: 000 | Epoch: 001 | Batch: 500 | Loss: 1.74478 | Correct: 312/512
Estimator: 000 | Epoch: 001 | Batch: 600 | Loss: 1.78413 | Correct: 293/512
Estimator: 000 | Epoch: 001 | Batch: 700 | Loss: 1.61722 | Correct: 339/512
Estimator: 000 | Epoch: 001 | Batch: 800 | Loss: 1.65374 | Correct: 332/512
Estimator: 000 | Epoch: 001 | Batch: 900 | Loss: 1.67443 | Correct: 321/512
Estimator: 000 | Epoch: 001 | Batch: 1000 | Loss: 1.71028 | Correct: 333/512
Estimator: 000 | Epoch: 001 | Batch: 1100 | Loss: 1.70154 | Correct: 328/512
Estimator: 000 | Epoch: 001 | Batch: 1200 | Loss: 1.74436 | Correct: 319/512
Estimator

2023-03-17 04:36:38,686 - INFO: Saving the model to `./VotingClassifier_Classifier_5_ckpt.pth`
2023-03-17 04:36:38,904 - INFO: Epoch: 001 | Validation Acc: 70.808 % | Historical Best: 70.808 %


Estimator: 000 | Epoch: 002 | Batch: 000 | Loss: 1.68038 | Correct: 325/512
Estimator: 000 | Epoch: 002 | Batch: 100 | Loss: 1.56562 | Correct: 356/512
Estimator: 000 | Epoch: 002 | Batch: 200 | Loss: 1.63043 | Correct: 329/512
Estimator: 000 | Epoch: 002 | Batch: 300 | Loss: 1.55221 | Correct: 343/512
Estimator: 000 | Epoch: 002 | Batch: 400 | Loss: 1.62874 | Correct: 330/512
Estimator: 000 | Epoch: 002 | Batch: 500 | Loss: 1.57668 | Correct: 342/512
Estimator: 000 | Epoch: 002 | Batch: 600 | Loss: 1.64223 | Correct: 325/512
Estimator: 000 | Epoch: 002 | Batch: 700 | Loss: 1.59091 | Correct: 343/512
Estimator: 000 | Epoch: 002 | Batch: 800 | Loss: 1.64411 | Correct: 327/512
Estimator: 000 | Epoch: 002 | Batch: 900 | Loss: 1.71426 | Correct: 314/512
Estimator: 000 | Epoch: 002 | Batch: 1000 | Loss: 1.65810 | Correct: 323/512
Estimator: 000 | Epoch: 002 | Batch: 1100 | Loss: 1.56866 | Correct: 342/512
Estimator: 000 | Epoch: 002 | Batch: 1200 | Loss: 1.60311 | Correct: 339/512
Estimator

...

2023-03-17 05:25:44,561 - INFO: Saving the model to `./VotingClassifier_Classifier_5_ckpt.pth`
2023-03-17 05:25:44,785 - INFO: Epoch: 017 | Validation Acc: 82.421 % | Historical Best: 82.421 %


Estimator: 000 | Epoch: 018 | Batch: 000 | Loss: 1.24584 | Correct: 401/512
Estimator: 000 | Epoch: 018 | Batch: 100 | Loss: 1.35962 | Correct: 370/512
Estimator: 000 | Epoch: 018 | Batch: 200 | Loss: 1.26440 | Correct: 406/512
Estimator: 000 | Epoch: 018 | Batch: 300 | Loss: 1.38952 | Correct: 377/512
Estimator: 000 | Epoch: 018 | Batch: 400 | Loss: 1.37910 | Correct: 374/512
Estimator: 000 | Epoch: 018 | Batch: 500 | Loss: 1.33315 | Correct: 387/512
Estimator: 000 | Epoch: 018 | Batch: 600 | Loss: 1.35904 | Correct: 383/512
Estimator: 000 | Epoch: 018 | Batch: 700 | Loss: 1.28505 | Correct: 390/512
Estimator: 000 | Epoch: 018 | Batch: 800 | Loss: 1.25709 | Correct: 394/512
Estimator: 000 | Epoch: 018 | Batch: 900 | Loss: 1.32734 | Correct: 390/512
Estimator: 000 | Epoch: 018 | Batch: 1000 | Loss: 1.25177 | Correct: 393/512
Estimator: 000 | Epoch: 018 | Batch: 1100 | Loss: 1.30937 | Correct: 383/512
Estimator: 000 | Epoch: 018 | Batch: 1200 | Loss: 1.31865 | Correct: 387/512
Estimator

2023-03-17 05:28:48,051 - INFO: Saving the model to `./VotingClassifier_Classifier_5_ckpt.pth`
2023-03-17 05:28:48,307 - INFO: Epoch: 018 | Validation Acc: 82.755 % | Historical Best: 82.755 %


Estimator: 000 | Epoch: 019 | Batch: 000 | Loss: 1.29456 | Correct: 395/512
Estimator: 000 | Epoch: 019 | Batch: 100 | Loss: 1.27116 | Correct: 389/512
Estimator: 000 | Epoch: 019 | Batch: 200 | Loss: 1.38865 | Correct: 377/512
Estimator: 000 | Epoch: 019 | Batch: 300 | Loss: 1.30256 | Correct: 389/512
Estimator: 000 | Epoch: 019 | Batch: 400 | Loss: 1.40639 | Correct: 370/512
Estimator: 000 | Epoch: 019 | Batch: 500 | Loss: 1.37225 | Correct: 374/512
Estimator: 000 | Epoch: 019 | Batch: 600 | Loss: 1.30800 | Correct: 393/512
Estimator: 000 | Epoch: 019 | Batch: 700 | Loss: 1.36777 | Correct: 370/512
Estimator: 000 | Epoch: 019 | Batch: 800 | Loss: 1.34004 | Correct: 382/512
Estimator: 000 | Epoch: 019 | Batch: 900 | Loss: 1.34960 | Correct: 386/512
Estimator: 000 | Epoch: 019 | Batch: 1000 | Loss: 1.28487 | Correct: 386/512
Estimator: 000 | Epoch: 019 | Batch: 1100 | Loss: 1.36817 | Correct: 377/512
Estimator: 000 | Epoch: 019 | Batch: 1200 | Loss: 1.35599 | Correct: 384/512
Estimator

2023-03-17 05:31:50,918 - INFO: Saving the model to `./VotingClassifier_Classifier_5_ckpt.pth`
2023-03-17 05:31:51,136 - INFO: Epoch: 019 | Validation Acc: 83.044 % | Historical Best: 83.044 %


# Training

In [9]:
del train_set, val_set
del train_loader, val_loader
gc.collect()

45

# Testing
Create a testing dataset, and load model from the saved checkpoint.

In [10]:
# load data
test_X = preprocess_data(split='test', feat_dir='/kaggle/input/ml2023spring-hw2/libriphone/feat', phone_path='/kaggle/input/ml2023spring-hw2/libriphone', concat_nframes=concat_nframes)
test_set = LibriDataset(test_X, None)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

[Dataset] - # phone classes: 41, number of utterances for test: 857


857it [00:07, 111.80it/s]

[INFO] test set
torch.Size([527364, 975])


In [11]:
# load model
from torchensemble.utils import io
test_ensemble = VotingClassifier(
    estimator=Classifier(input_dim=input_dim, hidden_layers=hidden_layers, hidden_dim=hidden_dim),               # here is your deep learning model
    n_estimators=estimators,                        # number of base estimators
    cuda=True,
)
io.load(test_ensemble,'./')  # reload


Make prediction.

In [12]:


pred = np.array([], dtype=np.int32)


with torch.no_grad():
    for i, batch in enumerate(tqdm(test_loader)):
        features = batch
        features = features.to(device)
        
        outputs = test_ensemble.predict(features)

        _, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        pred = np.concatenate((pred, test_pred.cpu().numpy()), axis=0)


100%|██████████| 1031/1031 [00:08<00:00, 126.01it/s]


Write prediction to a CSV file.

After finish running this block, download the file `prediction.csv` from the files section on the left-hand side and submit it to Kaggle.

In [13]:
with open('prediction.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(pred):
        f.write('{},{}\n'.format(i, y))